# "Road to recovery"

Simulation of various control measures under the asumption of remaining exposed (infected but not infectious) within the community.Arbitrayr epidemic parameters are presumed and then model with different network structures which accurately reflect different levels of "work-from-home", restrictions on gatherings, and/or COVIDSafe adoption - *under the assumption of those arbitrary choces of epidemic parameters*.

First, load some libraries....

In [1]:
cd("/Users/michael/work/GitHub/epinets")

In [27]:
include("EpiSim.jl")

Main.EpiSim

In [3]:
using CSV
using Plots
using LightGraphs
using JLD2, FileIO
using Statistics

Population of WA

In [4]:
pops=[426709 8089526 245869 5095100 1751693 534281 6594804 2621680]
pop=pops[8]

2621680

## Parameter guesstimates. 

Note, unlike previous simulations we only care about one choice of each parameter - there is no changepoint here.

In [5]:
#'reasonable' parameters
epiparam=Dict()
epiparam["p0"]=1/10 #a guess - tuned to match observed data 
epiparam["q"]=1/8 #"up to" two weeks
epiparam["r0"]=1/4 #about two weeks for mild, 3-6 for severe
epiparam["nseeds"]=5 #probably too many, consider dropping.
#parameters don't change across transition point - assume about 5 undiagnosed case 
#and control the covidsafe uptake via network structure
i=8
epiparam["pop"]=Int(floor(sqrt(pops[i])))^2
epiparam["gridsize"]=Int(floor(sqrt(pops[i])))

1619

## Build sample contact networks


Build contact networks with no restrictions and mass gatherings (bamodel), purely local (lattice), some level of work-from-home (wattsrog), or random mixing but no mass gatherings (randomgraph)

In [6]:
gridsize=epiparam["gridsize"]
bamodel=barabasi_albert(gridsize^2, 3, 2)
lattice=LightGraphs.grid((gridsize,gridsize),periodic=true)
#"social distancing"
wattstrog95=watts_strogatz(gridsize^2, 4, 0.013)  #s=0.013 => 95% compliance
wattstrog90=watts_strogatz(gridsize^2, 4, 0.026)  #s=0.026 => 90% compliance
wattstrog80=watts_strogatz(gridsize^2, 4, 0.053)  #s=0.053 => 80% compliance
wattstrog60=watts_strogatz(gridsize^2, 4, 0.120)  #s=0.120 => 60% compliance
wattstrog40=watts_strogatz(gridsize^2, 4, 0.205)  #s=0.205 => 40% compliance
wattstrog20=watts_strogatz(gridsize^2, 4, 0.332)  #s=0.332 => 20% compliance
randomgraph=watts_strogatz(gridsize^2, 4, 1)  #0% compliance

{2621161, 5242322} undirected simple Int64 graph

To simulate the effect of COVIDSafe, we remove links corresponding to the contacts that would be traced with the app. That is, if k is the fraction with the app, then we deletec k^2 edges from our graph.

In [7]:
#"covidsafe" at 40%
covidsafe=0.4
iso=covidsafe^2 #covidsafe implies removing iso fraction of all edges of graph
covidsafe40=deepcopy(randomgraph) 
for edg in edges(covidsafe40)
    if rand(Float64) .< iso
        rem_edge!(covidsafe40,edg)
    end
end
#there are two parameters to play with here - the underlying model (here bamodel, but it could be an ER graph, or a truncated
#scale free network, or a lattice - wattsrog - with a certain fraction of remote links) The second parameter is the level of covidsafe adoption
# of course the original transmission parameters could be changed too....

In [8]:
#"covidsafe" at 60%
covidsafe=0.6
iso=covidsafe^2 #covidsafe implies removing iso fraction of all edges of graph
covidsafe60=deepcopy(wattstrog40) 
for edg in edges(covidsafe60)
    if rand(Float64) .< iso
        rem_edge!(covidsafe60,edg)
    end
end
#40% work from home (wattstrog40 plus 60% COVIDSafe)

In [24]:
#"covidsafe" at 60%
covidsafe=0.5
iso=covidsafe^2 #covidsafe implies removing iso fraction of all edges of graph
safe50home=deepcopy(wattstrog40) 
for edg in edges(safe50home)
    if rand(Float64) .< iso
        rem_edge!(safe50home,edg)
    end
end
#40% work from home (wattstrog40 plus 50% COVIDSafe)

In [9]:
#"covidsafe" at 80%
covidsafe=0.8
iso=covidsafe^2 #covidsafe implies removing iso fraction of all edges of graph
covidsafe80=deepcopy(bamodel) 
for edg in edges(covidsafe80)
    if rand(Float64) .< iso
        rem_edge!(covidsafe80,edg)
    end
end
#40% work from home (wattstrog40 plus 60% COVIDSafe)

In [10]:
#"covidsafe" at 50% + footy, 50 people or no gatherings
covidsafe=0.5
iso=covidsafe^2 #covidsafe implies removing iso fraction of all edges of graph
safe50footy=deepcopy(bamodel) 
for edg in edges(safe50footy)
    if rand(Float64) .< iso
        rem_edge!(safe50footy,edg)
    end
end
safe50nmass=deepcopy(randomgraph) 
for edg in edges(safe50nmass)
    if rand(Float64) .< iso
        rem_edge!(safe50nmass,edg)
    end
end

To simulate restrictions on mass gatherings, we take the BA model and delete edges from high degree nodes

In [11]:
#No more than 50 people
gather=50
gathering50=deepcopy(bamodel)
for nds in vertices(gathering50)
    neigh=neighbors(gathering50,nds)
    while length(neigh)>gather
        rem_edge!(gathering50,nds,rand(neigh))
        neigh=neighbors(gathering50,nds) 
    end
end

In [12]:
#combined: "covidsafe" at 50% - and no more than 50 people.
covidsafe=0.5
iso=covidsafe^2 #covidsafe implies removing iso fraction of all edges of graph
covidsafe50=deepcopy(gathering50) 
for edg in edges(covidsafe50)
    if rand(Float64) .< iso
        rem_edge!(covidsafe50,edg)
    end
end
safe50no50=deepcopy(covidsafe50)

{2621161, 3895439} undirected simple Int64 graph

## Simulations

Now we run a bunch of simualtions with these presumed control parameters and the network contact graphs generated above

In [13]:
ndays=150#120 days, 100 simulations
nsims=100
St0,Et0,It0,Rt0=EpiSim.episim(bamodel, epiparam, ndays, nsims)   #no-limits - back to ther footy
St1,Et1,It1,Rt1=EpiSim.episim(covidsafe80, epiparam, ndays, nsims) #COVIDSafe at 80%, no other restrictions
St2,Et2,It2,Rt2=EpiSim.episim(randomgraph, epiparam, ndays, nsims) #no mass gatherings
St3,Et3,It3,Rt3=EpiSim.episim(covidsafe40, epiparam, ndays, nsims) #no mass gatherings, and 40% COVIDSafe
St4,Et4,It4,Rt4=EpiSim.episim(wattstrog40, epiparam, ndays, nsims) #40% social isolation
St5,Et5,It5,Rt5=EpiSim.episim(covidsafe60, epiparam, ndays, nsims) #40% social isolation & 60% COVIDSafe
St6,Et6,It6,Rt6=EpiSim.episim(gathering50, epiparam, ndays, nsims) #no more than 50 people
St7,Et7,It7,Rt7=EpiSim.episim(covidsafe50, epiparam, ndays, nsims) #no more than 50 people, COVIDSafe at 50%



100.0%┣████████████████████████████████████████┫ 100/100 [27:20<00:00, 0.1 it/s]
100.0%┣████████████████████████████████████████┫ 100/100 [17:11<00:00, 0.1 it/s]
100.0%┣████████████████████████████████████████┫ 100/100 [42:52<00:00, 0.0 it/s]
100.0%┣████████████████████████████████████████┫ 100/100 [21:17<00:00, 0.1 it/s]
100.0%┣████████████████████████████████████████┫ 100/100 [27:04<00:00, 0.1 it/s]
100.0%┣████████████████████████████████████████┫ 100/100 [15:33<00:00, 0.1 it/s]
100.0%┣████████████████████████████████████████┫ 100/100 [20:05<00:00, 0.1 it/s]
100.0%┣████████████████████████████████████████┫ 100/100 [18:47<00:00, 0.1 it/s]


(UInt64[0x000000000027fee4 0x000000000027fee4 … 0x000000000027fee4 0x000000000027fee4; 0x000000000027fee4 0x000000000027fee4 … 0x000000000027fee3 0x000000000027fee4; … ; 0x000000000027fedb 0x000000000027fec9 … 0x000000000027fecc 0x000000000027fdf4; 0x000000000027fedb 0x000000000027fec9 … 0x000000000027fecc 0x000000000027fdee], UInt64[0x0000000000000005 0x0000000000000004 … 0x0000000000000002 0x0000000000000005; 0x0000000000000004 0x0000000000000004 … 0x0000000000000003 0x0000000000000004; … ; 0x0000000000000000 0x0000000000000000 … 0x0000000000000000 0x000000000000002c; 0x0000000000000000 0x0000000000000000 … 0x0000000000000000 0x000000000000002f], UInt64[0x0000000000000000 0x0000000000000001 … 0x0000000000000003 0x0000000000000000; 0x0000000000000001 0x0000000000000000 … 0x0000000000000003 0x0000000000000001; … ; 0x0000000000000000 0x0000000000000000 … 0x0000000000000000 0x0000000000000011; 0x0000000000000000 0x0000000000000000 … 0x0000000000000000 0x000000000000000f], UInt64[0x000000

## Plot

In [14]:
compl=0.2
swtch=1-compl^(1/4)
plotly()

┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots /Users/michael/.julia/packages/Plots/cc8wh/src/backends.jl:363


Plots.PlotlyBackend()

In [15]:
pop=epiparam["pop"]
plot([],title="Total anticipated infections",xlabel="day",ylabel="Infections",ylimit=(0,500),label=false,size=(1600,800))
EpiSim.plotquantiles(pop .- St0,:black,"Back to the footy",0.25)
EpiSim.plotquantiles(pop .- St1,:red,"Footy & 80% COVIDSafe",0.25)
EpiSim.plotquantiles(pop .- St6,:purple,"50 person limit",0.25)
EpiSim.plotquantiles(pop .- St7,:magenta,"50 person limit & 50% COVIDSafe",0.25)
EpiSim.plotquantiles(pop .- St2,:yellow,"No mass gatherings",0.25)
EpiSim.plotquantiles(pop .- St3,:orange,"40% COVIDSafe",0.25)
EpiSim.plotquantiles(pop .- St4,:blue,"40% Work-from-home",0.25)
EpiSim.plotquantiles(pop .- St5,:green,"40% Work-from-home & 60% COVIDSafe",0.25)

([5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.75, 6.0, 6.0, 6.0  …  9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0], [5.0, 5.0, 5.0, 5.0, 6.0, 6.0, 6.0, 7.0, 7.0, 8.0  …  12.5, 12.5, 12.5, 12.5, 12.5, 12.5, 12.5, 12.5, 12.5, 12.5], [5.0, 5.0, 6.0, 6.0, 7.0, 7.0, 7.0, 8.0, 9.0, 9.0  …  18.0, 18.0, 18.0, 18.0, 18.0, 18.0, 18.0, 18.0, 18.0, 18.0])

One I prepared earlier ...  with p=1/10, r=16.

In [16]:
plot([],title="Total anticipated infections",xlabel="day",ylabel="Infections",ylimit=(0,5000),label=false,size=(1200,800))
EpiSim.plotquantiles(pop .- St0,:black,"Back to the footy",0.25)
EpiSim.plotquantiles(pop .- St1,:red,"Footy & 80% COVIDSafe",0.25)
EpiSim.plotquantiles(pop .- St2,:yellow,"No mass gatherings",0.25)
EpiSim.plotquantiles(pop .- St3,:orange,"40% COVIDSafe",0.25)
EpiSim.plotquantiles(pop .- St4,:blue,"40% Work-from-home",0.25)
EpiSim.plotquantiles(pop .- St5,:green,"40% Work-from-home & 60% COVIDSafe",0.25)

([5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.75, 6.0, 6.0, 6.0  …  9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0], [5.0, 5.0, 5.0, 5.0, 6.0, 6.0, 6.0, 7.0, 7.0, 8.0  …  12.5, 12.5, 12.5, 12.5, 12.5, 12.5, 12.5, 12.5, 12.5, 12.5], [5.0, 5.0, 6.0, 6.0, 7.0, 7.0, 7.0, 8.0, 9.0, 9.0  …  18.0, 18.0, 18.0, 18.0, 18.0, 18.0, 18.0, 18.0, 18.0, 18.0])

One I prepared earlier ...  with p=1/6, r=16.

In [22]:
plot([],title="Total anticipated infections",xlabel="day",ylabel="Infections",ylimit=(0,5000),label=false,size=(1200,800))
EpiSim.plotquantiles(pop .- St0,:black,"Back to the footy",0.25)
EpiSim.plotquantiles(pop .- St1,:red,"Footy & 80% COVIDSafe",0.25)
EpiSim.plotquantiles(pop .- St2,:yellow,"No mass gatherings",0.25)
EpiSim.plotquantiles(pop .- St3,:orange,"40% COVIDSafe",0.25)
EpiSim.plotquantiles(pop .- St4,:blue,"40% Work-from-home",0.25)
EpiSim.plotquantiles(pop .- St5,:green,"40% Work-from-home & 60% COVIDSafe",0.25)

<!DOCTYPE html>
 
 
 Plots.jl

In [18]:
plot()
EpiSim.plotquantiles(pop .- St7,:magenta,"50 person limit & 50% COVIDSafe",0.25)

([5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0  …  12.75, 12.75, 12.75, 12.75, 12.75, 12.75, 12.75, 12.75, 12.75, 12.75], [5.0, 5.0, 5.0, 5.0, 5.0, 6.0, 6.0, 6.0, 6.0, 7.0  …  253.5, 258.5, 262.0, 269.0, 274.5, 285.5, 297.0, 305.5, 314.0, 318.5], [5.0, 5.0, 5.25, 6.0, 6.0, 7.0, 8.0, 8.0, 9.0, 9.0  …  2427.0, 2493.25, 2563.0, 2647.75, 2742.25, 2846.5, 2971.25, 3112.0, 3239.0, 3385.75])

In [19]:
#@save "wsj_part1.jld"


In [25]:
St8,Et8,It8,Rt8=EpiSim.episim(safe50footy, epiparam, ndays, nsims) #footy, COVIDSafe at 50%
St9,Et9,It9,Rt9=EpiSim.episim(safe50nmass, epiparam, ndays, nsims) #no more than 50 people, COVIDSafe at 50%
St10,Et10,It10,Rt10=EpiSim.episim(safe50home, epiparam, ndays, nsims) #stay home, COVIDSafe at 50%

100.0%┣████████████████████████████████████████┫ 100/100 [20:43<00:00, 0.1 it/s]
100.0%┣████████████████████████████████████████┫ 100/100 [23:13<00:00, 0.1 it/s]
100.0%┣████████████████████████████████████████┫ 100/100 [20:24<00:00, 0.1 it/s]


(UInt64[0x000000000027fee4 0x000000000027fee4 … 0x000000000027fee4 0x000000000027fee4; 0x000000000027fee4 0x000000000027fee4 … 0x000000000027fee4 0x000000000027fee4; … ; 0x000000000027fed7 0x000000000027fedf … 0x000000000027feda 0x000000000027fee0; 0x000000000027fed7 0x000000000027fedf … 0x000000000027feda 0x000000000027fee0], UInt64[0x0000000000000005 0x0000000000000004 … 0x0000000000000005 0x0000000000000003; 0x0000000000000005 0x0000000000000004 … 0x0000000000000004 0x0000000000000002; … ; 0x0000000000000000 0x0000000000000000 … 0x0000000000000000 0x0000000000000000; 0x0000000000000000 0x0000000000000000 … 0x0000000000000000 0x0000000000000000], UInt64[0x0000000000000000 0x0000000000000001 … 0x0000000000000000 0x0000000000000002; 0x0000000000000000 0x0000000000000001 … 0x0000000000000001 0x0000000000000002; … ; 0x0000000000000000 0x0000000000000000 … 0x0000000000000000 0x0000000000000000; 0x0000000000000000 0x0000000000000000 … 0x0000000000000000 0x0000000000000000], UInt64[0x000000

In [31]:
(l1,m1,h1)=EpiSim.plotquantiles(pop .- St0,:black,"Permit mass gathering",0.25)
(l2,m2,h2)=EpiSim.plotquantiles(pop .- St8,:purple,"Mass gathering & contact tracing",0.25)
(l3,m3,h3)=EpiSim.plotquantiles(pop .- St6,:purple,"50 person limit",0.25)
(l4,m4,h4)=EpiSim.plotquantiles(pop .- St7,:magenta,"50 person limit & contact tracing",0.25)
(l5,m5,h5)=EpiSim.plotquantiles(pop .- St2,:yellow,"No mass gatherings",0.25)
(l6,m6,h6)=EpiSim.plotquantiles(pop .- St9,:yellow,"No mass & contact tracing",0.25)
(l7,m7,h7)=EpiSim.plotquantiles(pop .- St4,:yellow,"40% Stay home",0.25)
(l8,m8,h8)=EpiSim.plotquantiles(pop .- St10,:yellow,"40% Stay home & contact tracing",0.25)




([5.0, 5.0, 5.0, 5.0, 5.0, 6.0, 6.0, 6.0, 6.0, 7.0  …  11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0], [5.0, 5.0, 5.0, 6.0, 6.0, 6.0, 7.0, 7.0, 8.0, 8.0  …  15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0], [5.0, 5.0, 6.0, 7.0, 7.0, 8.0, 8.0, 9.0, 9.0, 9.0  …  23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0])

In [55]:
upscale = 1.3
fntsm = Plots.font("sans-serif", pointsize=round(10.0*upscale))
fntlg = Plots.font("sans-serif", pointsize=round(14.0*upscale))
default(titlefont=fntlg, guidefont=fntlg, tickfont=fntsm, legendfont=fntsm)


In [71]:
labels=["mass gathering" "mass gath. with CT" "50 person limit" "50 ppl. with CT" "no mass gatherings" "no gath. with CT" "40% stay home" "40% with CT" ]
linetyp=[:solid :dash :solid :dash :solid :dash :solid :dash ]
linecol=[:black :black :red :red :blue :blue :green :green]
plot([m1 m2 m3 m4 m5 m6 m7 m8],yaxis=:log,line=linetyp,color=linecol,label=labels,linewidth=3,xlabel="Days",ylabel="Infected",title="p=0.1, q=0.125, r=0.25",size=(1000,1200))




<!DOCTYPE html>
 
 
 Plots.jl

In [65]:
labels[3]

"50 person limit"